### Working with Google API to extract data on restaurants

In [3]:
# api_key = read in api key from separate file

In [5]:
import requests
import json

In [6]:
query = "restaurants near boston university"
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=%s&key=%s" %(query, api_key)

try:
    response = requests.get(url)
    if response.status_code != 200:
        print('HTTP error', response.status_code)
    else:
        try:
            response_data = response.json()
        except:
            print("response not valid JSON format")
except:
    print("something went wrong with requests.get")

In [40]:
response_data.keys()
print(type(response_data['results'][0]))

<class 'dict'>


In [36]:
for key in response_data['results']:
    print(key)

{'business_status': 'OPERATIONAL', 'formatted_address': '712 Commonwealth Avenue, Boston, MA 02215, United States', 'geometry': {'location': {'lat': 42.3495484, 'lng': -71.1055686}, 'viewport': {'northeast': {'lat': 42.35089822989272, 'lng': -71.10421877010728}, 'southwest': {'lat': 42.34819857010728, 'lng': -71.10691842989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': 'University grill & pizza', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3264, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113218831869395214706">M Burnett</a>'], 'photo_reference': 'Aap_uEDa_SzXYvvpLxiMrdi3nbMUGJ27In-RIFnth_bOcA2t4Z9RQQn2_njIMQa9jZbpSMwUgLs6CbF33bqc9ZWHIs2iV15S7j4XxM8dshv62cMQdZwhrVAtUzZBUCHZoUXutZwHnFA2awhHn1P2SZKBcv5MQoGrbVp0O0NqvvqPBJpKz15v', 'width': 2448}], 'place_id': 'Ch

In [21]:

for i in range(len(response_data['results'])):
    restaurant = response_data['results'][i]
    

{'business_status': 'OPERATIONAL', 'formatted_address': '712 Commonwealth Avenue, Boston, MA 02215, United States', 'geometry': {'location': {'lat': 42.3495484, 'lng': -71.1055686}, 'viewport': {'northeast': {'lat': 42.35089822989272, 'lng': -71.10421877010728}, 'southwest': {'lat': 42.34819857010728, 'lng': -71.10691842989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': 'University grill & pizza', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3264, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113218831869395214706">M Burnett</a>'], 'photo_reference': 'Aap_uEDa_SzXYvvpLxiMrdi3nbMUGJ27In-RIFnth_bOcA2t4Z9RQQn2_njIMQa9jZbpSMwUgLs6CbF33bqc9ZWHIs2iV15S7j4XxM8dshv62cMQdZwhrVAtUzZBUCHZoUXutZwHnFA2awhHn1P2SZKBcv5MQoGrbVp0O0NqvvqPBJpKz15v', 'width': 2448}], 'place_id': 'Ch

In [11]:
def get_businesses(query,api_key, min_rating=0.0, max_results = 0):
    """Takes the parameters: query, api_key, min_rating, max_results
        query: query parameter is a string ie("restaurants near Columbia Univeristy"
        api_key: key used to call the data in Google's Places API
        min_rating=: optional argument is an integer including 1.0, 2.0, 3.0 or 4.0 that returns only places with a 
                    rating greater than min_rating
        max_results=: integer that will return maximum of the max_results. ie(20))"""
    import requests
    import json
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=%s&key=%s" %(query, api_key)

    try:
        response = requests.get(url)
        if response.status_code != 200:
            print("HTTP error", response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
            print("Something went wrong with requests.get")

    result = []
    for i in range(len(response_data['results'])):
        restaurant = response_data['results'][i]
        if restaurant['rating'] > min_rating:
            name = restaurant['name']
            formatted_address = restaurant['formatted_address']
            open_now = restaurant['opening_hours']
            try:
                price_level = restaurant['price_level']
            except:
                price_level = 'None'
            rating = restaurant['rating']

            result.append((name, formatted_address, open_now, price_level, rating))

    while len(result) < max_results and 'next_page_token' in response_data:
        token = response_data['next_page_token']
        new_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken=%s&key=%s" %(token, api_key)

        response = requests.get(new_url)     
        while response.json().get('status') != 'OK':
            from time import sleep
            from random import random
            sleep(random())
            response = requests.get(new_url)
        
        response_data = response.json()

        for i in range(len(response_data['results'])):
            restaurant = response_data['results'][i]
            if restaurant['rating'] > min_rating:
                name = restaurant['name']
                formatted_address = restaurant['formatted_address']
                open_now = restaurant['opening_hours']
                try:
                    price_level = restaurant['price_level']
                except:
                    price_level = 'None'
                rating = restaurant['rating']

                result.append((name, formatted_address, open_now, price_level, rating))

    if max_results > 0 :
        return result[:max_results]
    else:
        return result

In [16]:
get_businesses('restaurants near Columbia University', api_key, min_rating=3, max_results=10)

[('Community Food & Juice',
  '2893 Broadway, New York, NY 10025, United States',
  {'open_now': True},
  2,
  4.3),
 ('Cafe Du Soleil',
  '2723 Broadway, New York, NY 10025, United States',
  {'open_now': True},
  2,
  4.2),
 ('Junzi Kitchen - Morningside Heights',
  '2896 Broadway, Manhattan, NY 10025, United States',
  {'open_now': True},
  2,
  4.3),
 ('Tartina',
  '1034 Amsterdam Ave, New York, NY 10025, United States',
  {'open_now': True},
  2,
  4.4),
 ('John Jay Dining Hall',
  '519 W 114th St, New York, NY 10027, United States',
  {'open_now': True},
  'None',
  4.2),
 ('Symposium',
  '544 W 113th St, New York, NY 10025, United States',
  {'open_now': True},
  2,
  4.2),
 ('sweetgreen',
  '2937 Broadway, New York, NY 10025, United States',
  {'open_now': True},
  2,
  4),
 ('Massawa',
  '1239 Amsterdam Ave, New York, NY 10027, United States',
  {'open_now': True},
  2,
  4.5),
 ('Le Monde',
  '2885 Broadway, New York, NY 10025, United States',
  {'open_now': True},
  2,
  4),